In [2]:
import json
from algorithm.data import get_data
tr_data, dev_data, test_data = get_data()
total_data = {}
total_data.update(tr_data)
total_data.update(dev_data)
total_data.update(test_data)
sents = []

In [3]:
ent_map = {}
with open("kg_embed/entity_map.txt") as fin:
    for line in fin:
        name, qid = line.strip().split("\t")
        ent_map[name] = qid
entity2id = {}
with open("kg_embed/entity2id.txt") as fin:
    fin.readline()
    for line in fin:
        qid, eid = line.strip().split('\t')
        entity2id[qid] = int(eid)
with open("data/entities.txt") as fin:
    ents = json.load(fin)

In [4]:
from collections import defaultdict
temp = defaultdict(set)
for k,v in ents.items():
    for e in v:
        if e in ent_map:
            temp[k].add(ent_map[e])
temp = dict(temp)
ent_ids = defaultdict(set)
for k,v in temp.items():
    for e in v:
        if e in entity2id:
            ent_ids[k].add(entity2id[e])
ent_ids = dict(ent_ids)

In [18]:
del(ent_map)
del(entity2id)
del(temp)
vecs = []
with open("kg_embed/entity2vec.vec", 'r') as fin:
    for line in fin:
        vec = line.strip().split('\t')
        vec = [float(x) for x in vec]
        vecs.append(vec)

In [22]:
ent_embed = defaultdict(list)
for k,v in ent_ids.items():
    for eid in v:
        ent_embed[k].append(vecs[eid])
ent_embed = dict(ent_embed)

In [26]:
with open('data/entity_embed.json', 'w+') as f:
    json.dump(ent_embed, f, indent=4)

In [27]:
from algorithm.data import get_data, LABEL_FREQ
import json
ent_map = {}
with open("kg_embed/entity_map.txt") as fin:
    for line in fin:
        name, qid = line.strip().split("\t")
        ent_map[name] = qid

def get_ents(ann):
    ents = []
    # Keep annotations with a score higher than 0.3
    for a in ann.get_annotations(0.3):
        if a.entity_title not in ent_map:
            continue
        ents.append([ent_map[a.entity_title], a.begin, a.end, a.score])
    return ents

tr_data, dev_data, test_data = get_data()
total_data = {}
total_data.update(tr_data)
total_data.update(dev_data)
total_data.update(test_data)

In [29]:
import tagme
tagme.GCUBE_TOKEN = "<TagMe token here>"

In [30]:
data = {}
for k,v in total_data.items():
    sent = v[1]
    sent = sent.replace('\n\n', '\n')
    if sent.endswith('\n'):
        sent = sent[:-1]
    sent = sent.replace('\n', '\n ')
    d = {}
    d['label'] = v[0]
    d['text'] = sent
    d['entities'] = get_ents(tagme.annotate(sent))
    data[k] = d

In [ ]:
with open('data/entity_preprocessed.json', 'w+') as f:
    json.dump(data, f, indent=4)